In [2]:
!bash /home/azureuser/cloudfiles/code/blobfuse/blobfuse_raadsinformatie.sh

In [3]:
pip install protobuf  

  Using cached protobuf-5.26.1-cp37-abi3-manylinux2014_x86_64.whl.metadata (592 bytes)
Using cached protobuf-5.26.1-cp37-abi3-manylinux2014_x86_64.whl (302 kB)
Note: you may need to restart the kernel to use updated packages.


In [5]:
import sys
sys.path.append("..")

# Select where to run notebook: "azure" or "local"
my_run = "azure"

import my_secrets as sc
import settings as st

if my_run == "azure":
    import config_azure as cf
elif my_run == "local":
    import config as cf


import os
if my_run == "azure":
    if not os.path.exists(cf.HUGGING_CACHE):
        os.mkdir(cf.HUGGING_CACHE)
    os.environ["TRANSFORMERS_CACHE"] = cf.HUGGING_CACHE


# setup environment GEITje-7B Finetuning
# - pip install torch
# - pip install datasets
# - pip install transformers
# - pip install trl
# - pip install accelerate (restart after)
# - switch device_map='auto' to avaoid memory error

# - pip install sentencepiece
# - pip install jupyter
# - pip install protobuf 


In [6]:
import torch
from datasets import DatasetDict, load_dataset, concatenate_datasets
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments
from trl import SFTTrainer


def train(model, tokenizer, chat_dataset, new_model_name):

    def format(examples):
        return [tokenizer.apply_chat_template(conversation, tokenize=False)
                for conversation in examples['messages_nl']]

    per_device_train_batch_size = 2
    gradient_accumulation_steps = 8
    steps_per_epoch = len(chat_dataset['train_sft'])\
                 // (torch.cuda.device_count() * per_device_train_batch_size * gradient_accumulation_steps)
    eval_steps = steps_per_epoch // 5

    training_args = TrainingArguments(
        optim='adamw_bnb_8bit',
        num_train_epochs=3,
        learning_rate=1e-5,
        lr_scheduler_type='cosine',
        warmup_ratio=0.1,
        per_device_train_batch_size=2,
        gradient_accumulation_steps=8,
        gradient_checkpointing=True,
        evaluation_strategy='steps',
        eval_steps=eval_steps,
        save_strategy='epoch',
        bf16=True,
        output_dir='/tmp/geitje/output',
        report_to=["tensorboard", 'wandb'],
        logging_steps=1,
        logging_first_step=True,
        hub_model_id=new_model_name,
        push_to_hub=True,
        hub_private_repo=True,
        hub_strategy='all_checkpoints',
    )

    trainer = SFTTrainer(
        model=model,
        args=training_args,
        tokenizer=tokenizer,
        max_seq_length=8192,
        train_dataset=chat_dataset['train_sft'],
        eval_dataset=chat_dataset['test_sft'],
        formatting_func=format,
        neftune_noise_alpha=5,
    )

    trainer.train()
    trainer.push_to_hub()


if __name__ == '__main__':
    basemodel_name = 'Rijgersberg/GEITje-7B'
    model = AutoModelForCausalLM.from_pretrained(basemodel_name, torch_dtype=torch.bfloat16,
                                                 low_cpu_mem_usage=True, attn_implementation="eager",
                                                 device_map='auto')
    tokenizer = AutoTokenizer.from_pretrained(basemodel_name)

    # Mistral 7B is missing a padding token by default, so we need to assign
    # another token to the padding job during training.
    # Unfortunately we cannot use the </s> token, because we need the model to
    # learn to output </s> at the end of its turn, so that we can stop generating
    # when it emits it. If we were to also use it as the padding token,
    # any loss computed on </s> would then be discarded, nothing would be learned
    # and the model would never stop generating.
    # Trust me, I learned this the hard way ;).
    # Therefore, we take the least bad alternative action and assign
    # the rarely used <UNK> token to the padding role.
    # tokenizer.pad_token = tokenizer.unk_token
    # model.config.pad_token_id = tokenizer.unk_token_id

    # no_robots_nl = load_dataset('Rijgersberg/no_robots_nl')
    # ultrachat_nl = load_dataset('Rijgersberg/ultrachat_10k_nl')

    # chat_dataset = DatasetDict({
    #     'train_sft': concatenate_datasets([no_robots_nl['train_sft'],
    #                                        ultrachat_nl['train_sft']]).shuffle(seed=42),
    #     'test_sft': concatenate_datasets([no_robots_nl['test_sft'],
    #                                       ultrachat_nl['test_sft']]).shuffle(seed=42),
    # })

    # chat_dataset = chat_dataset.filter(lambda row: all(turn['content'] != '<TRANSLATION FAILED>'
    #                                                    for turn in row['messages_nl']))

    # train(model, tokenizer, chat_dataset,
    #       new_model_name='Rijgersberg/GEITje-7B-chat')


/anaconda/envs/AmsterdamFinetuning/lib/python3.9/site-packages/accelerate/utils/modeling.py:1363: UserWarning: Current model requires 520097664 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [4]:
import torch
from datasets import DatasetDict, load_dataset, concatenate_datasets
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments
from trl import SFTTrainer

basemodel_name = 'Rijgersberg/GEITje-7B'
model = AutoModelForCausalLM.from_pretrained(basemodel_name, torch_dtype=torch.bfloat16,
                                                low_cpu_mem_usage=True, attn_implementation="eager",
                                                device_map='cpu',offload_buffers=True)
print("end model")
tokenizer = AutoTokenizer.from_pretrained(basemodel_name)
print('end tokenizer')
tokenizer.pad_token = tokenizer.unk_token
model.config.pad_token_id = tokenizer.unk_token_id


device = 'cuda' if torch.cuda.is_available() else 'cpu'

def generate_response(conversation, temperature=0.2, top_k=50, max_new_tokens=1_000):
    tokenized = tokenizer.apply_chat_template(conversation, add_generation_prompt=True,
                                              return_tensors='pt').to('cpu')
    outputs = model.generate(tokenized, do_sample=True, temperature=temperature,
                             top_k=top_k, max_new_tokens=max_new_tokens)
    print(outputs)

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

conversation = [
    {
        'role': 'user',
        'content': 'Welk woord hoort er niet in dit rijtje thuis: "auto, vliegtuig, geitje, bus"?'
    }
]
print(generate_response(conversation))

config.json:   0%|          | 0.00/668 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


end tokenizer


KeyboardInterrupt: 

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer


device = 'cuda' if torch.cuda.is_available() else 'cpu'

model_name = 'Rijgersberg/GEITje-7B-chat-v2'
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16,
                                             low_cpu_mem_usage=True, attn_implementation="eager",
                                             device_map=device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

def generate(conversation, temperature=0.2, top_k=50, max_new_tokens=1_000):
    tokenized = tokenizer.apply_chat_template(conversation, add_generation_prompt=True,
                                              return_tensors='pt').to(device)
    outputs = model.generate(tokenized, do_sample=True, temperature=temperature,
                             top_k=top_k, max_new_tokens=max_new_tokens)

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

conversation = [
    {
        'role': 'user',
        'content': 'Welk woord hoort er niet in dit rijtje thuis: "auto, vliegtuig, geitje, bus"?'
    }
]
print(generate(conversation))
# <|user|>
# Welk woord hoort er niet in dit rijtje thuis: "auto, vliegtuig, geitje, bus"? 
# <|assistant|>
# Het woord dat niet op zijn plaats staat is 'geit'. Een geit zou niet tussen een lijst van vervoersmiddelen moeten staan. Het past beter bij een boerderijthema of dierenlijst.

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 112.00 MiB. GPU 0 has a total capacity of 15.57 GiB of which 27.44 MiB is free. Process 22277 has 13.81 GiB memory in use. Including non-PyTorch memory, this process has 1.73 GiB memory in use. Of the allocated memory 1.58 GiB is allocated by PyTorch, and 25.84 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

def generate_response(conversation, temperature=0.2, top_k=50, max_new_tokens=250):
    tokenized = tokenizer.apply_chat_template(conversation, add_generation_prompt=True,
                                              return_tensors='pt').to(device)
    print(tokenized)
    outputs = model.generate(tokenized, do_sample=True, temperature=temperature,
                             top_k=top_k, max_new_tokens=max_new_tokens)

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

conversation = [
    {
        'role': 'user',
        'content': 'Welk woord hoort er niet in dit rijtje thuis: "auto, vliegtuig, geitje, bus"?'
    }
]
print(generate_response(conversation))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


tensor([[  523, 28766,  1838, 28766, 28767,    13, 28780,   301, 28729, 14277,
           556,  7463,   419,  1234, 14654,   297,  8052,  3191, 28768, 28707,
          2099,   306, 14867, 28747,   345,  6415, 28725,   363,  5715,  3606,
         28718,   326, 28725,  2970,   279,  2099, 28725,  1579, 27257,     2,
         28705,    13, 28789, 28766,   489, 11143, 28766, 28767,    13]],
       device='cuda:0')


KeyboardInterrupt: 

In [ ]:
pip install protobuf  

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.8/302.8 kB 9.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install flash-attn --no-build-isolation


  Using cached flash_attn-2.5.7.tar.gz (2.5 MB)
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [13 lines of output]
      fatal: not a git repository (or any parent up to mount point /)
      Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "/tmp/pip-install-0w7wgfd1/flash-attn_df563f3f7cd745e98d322f82a4b5cc97/setup.py", line 115, in <module>
          raise RuntimeError(
      RuntimeError: FlashAttention is only supported on CUDA 11.6 and above.  Note: make sure nvcc has a supported version by running nvcc -V.
      
      
      torch.__version__  = 2.2.2+cu121
      
      
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
err

In [ ]:
pip install cuda

ERROR: Could not find a version that satisfies the requirement cuda (from versions: none)
ERROR: No matching distribution found for cuda
Note: you may need to restart the kernel to use updated packages.


In [1]:
import sys
sys.path.append("..")

# Select where to run notebook: "azure" or "local"
my_run = "azure"

import my_secrets as sc
import settings as st

if my_run == "azure":
    import config_azure as cf
elif my_run == "local":
    import config as cf


import os
if my_run == "azure":
    if not os.path.exists(cf.HUGGING_CACHE):
        os.mkdir(cf.HUGGING_CACHE)
    os.environ["TRANSFORMERS_CACHE"] = cf.HUGGING_CACHE


In [6]:
import torch
from datasets import DatasetDict, load_dataset, concatenate_datasets
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments
from trl import SFTTrainer


def train(model, tokenizer, chat_dataset, new_model_name):

    def format(examples):
        return [tokenizer.apply_chat_template(conversation, tokenize=False)
                for conversation in examples['messages_nl']]

    per_device_train_batch_size = 2
    gradient_accumulation_steps = 8
    steps_per_epoch = len(chat_dataset['train_sft'])\
                 // (torch.cuda.device_count() * per_device_train_batch_size * gradient_accumulation_steps)
    eval_steps = steps_per_epoch // 5

    training_args = TrainingArguments(
        optim='adamw_bnb_8bit',
        num_train_epochs=3,
        learning_rate=1e-5,
        lr_scheduler_type='cosine',
        warmup_ratio=0.1,
        per_device_train_batch_size=2,
        gradient_accumulation_steps=8,
        gradient_checkpointing=True,
        evaluation_strategy='steps',
        eval_steps=eval_steps,
        save_strategy='epoch',
        bf16=True,
        tf32=True,
        output_dir='/tmp/geitje/output',
        report_to=["tensorboard", 'wandb'],
        logging_steps=1,
        logging_first_step=True,
        hub_model_id=new_model_name,
        push_to_hub=True,
        hub_private_repo=True,
        hub_strategy='all_checkpoints',
    )

    trainer = SFTTrainer(
        model=model,
        args=training_args,
        tokenizer=tokenizer,
        max_seq_length=8192,
        train_dataset=chat_dataset['train_sft'],
        eval_dataset=chat_dataset['test_sft'],
        formatting_func=format,
        neftune_noise_alpha=5,
    )

    trainer.train()
    trainer.push_to_hub()


if __name__ == '__main__':
    basemodel_name = 'Rijgersberg/GEITje-7B'
    model = AutoModelForCausalLM.from_pretrained(basemodel_name, torch_dtype=torch.bfloat16,
                                                 low_cpu_mem_usage=True, attn_implementation="sdpa",
                                                 device_map='auto')
    tokenizer = AutoTokenizer.from_pretrained(basemodel_name)

    # Mistral 7B is missing a padding token by default, so we need to assign
    # another token to the padding job during training.
    # Unfortunately we cannot use the </s> token, because we need the model to
    # learn to output </s> at the end of its turn, so that we can stop generating
    # when it emits it. If we were to also use it as the padding token,
    # any loss computed on </s> would then be discarded, nothing would be learned
    # and the model would never stop generating.
    # Trust me, I learned this the hard way ;).
    # Therefore, we take the least bad alternative action and assign
    # the rarely used <UNK> token to the padding role.
    tokenizer.pad_token = tokenizer.unk_token
    model.config.pad_token_id = tokenizer.unk_token_id

    no_robots_nl = load_dataset('Rijgersberg/no_robots_nl')
    no_robots_nl["train_sft"]=no_robots_nl["train_sft"].select(range(2))
    no_robots_nl["test_sft"]=no_robots_nl["test_sft"].select(range(2))

    ultrachat_nl = load_dataset('Rijgersberg/ultrachat_10k_nl')
    ultrachat_nl["train_sft"]=ultrachat_nl["train_sft"].select(range(2))
    ultrachat_nl["test_sft"]=ultrachat_nl["test_sft"].select(range(2))

    chat_dataset = DatasetDict({
        'train_sft': concatenate_datasets([no_robots_nl['train_sft'],
                                           ultrachat_nl['train_sft']]).shuffle(seed=42),
        'test_sft': concatenate_datasets([no_robots_nl['test_sft'],
                                          ultrachat_nl['test_sft']]).shuffle(seed=42),
    })

    chat_dataset = chat_dataset.filter(lambda row: all(turn['content'] != '<TRANSLATION FAILED>'
                                                       for turn in row['messages_nl']))

    train(model, tokenizer, chat_dataset,
          new_model_name='Rijgersberg/GEITje-7B-chat')

/anaconda/envs/AmsterdamFinetuning/lib/python3.9/site-packages/transformers/utils/import_utils.py:519: FutureWarning: `is_torch_tpu_available` is deprecated and will be removed in 4.41.0. Please use the `is_torch_xla_available` instead.
  


RuntimeError: Failed to import trl.trainer.sft_trainer because of the following error (look up to see its traceback):
No module named 'torch.distributed.algorithms'

In [5]:
pip install tokenizers==0.14

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 10.1 MB/s eta 0:00:00 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 13.2 MB/s eta 0:00:00
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.22.2
    Uninstalling huggingface-hub-0.22.2:
      Successfully uninstalled huggingface-hub-0.22.2
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.13.3
    Uninstalling tokenizers-0.13.3:
      Successfully uninstalled tokenizers-0.13.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
accelerate 0.29.1 requires torch>=1.10.0, but you have torch 1.7.1+cu110 which is incompatible.
datasets 2.18.0 requires huggingface-hub>=0.19.4, but you have huggingface-hub 0.16.4 which is incompatible.
transformers 4.28.0 requires tokenizers!=0.11.3,<0.14,>=0.11.1, but you have tokenizers 0.1

In [1]:
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu110

Looking in indexes: https://download.pytorch.org/whl/cu110
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 75.0 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 58.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 GB 1.3 MB/s eta 0:00:000:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 91.5 MB/s eta 0:00:00:00:01
INFO: pip is looking at multiple versions of torchaudio to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 63.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 68.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 64.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 107.7 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: torch
    Found existing installation: torch 2.2.2
    Uninstalling torch-2.2.2:
      Successfully

In [16]:
no_robots_nl = load_dataset('Rijgersberg/no_robots_nl')
no_robots_nl["train_sft"]=no_robots_nl["train_sft"].select(range(2))
no_robots_nl["test_sft"]=no_robots_nl["test_sft"].select(range(2))

ultrachat_nl = load_dataset('Rijgersberg/ultrachat_10k_nl')
ultrachat_nl["train_sft"]=ultrachat_nl["train_sft"].select(range(2))
ultrachat_nl["test_sft"]=ultrachat_nl["test_sft"].select(range(2))


chat_dataset = DatasetDict({
    'train_sft': concatenate_datasets([no_robots_nl['train_sft'],
                                        ultrachat_nl['train_sft']]).shuffle(seed=42),
    'test_sft': concatenate_datasets([no_robots_nl['test_sft'],
                                        ultrachat_nl['test_sft']]).shuffle(seed=42),
})

display(chat_dataset)


DatasetDict({
    train_sft: Dataset({
        features: ['prompt', 'prompt_id', 'messages', 'category', 'messages_nl'],
        num_rows: 4
    })
    test_sft: Dataset({
        features: ['prompt', 'prompt_id', 'messages', 'category', 'messages_nl'],
        num_rows: 4
    })
})